In [1]:
%load_ext autoreload
%autoreload 2

# %load_ext memory_profiler


In [2]:
# Check MPS device available
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
torch.set_default_device(mps_device)

In [4]:
from torchvision import datasets, transforms

    
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                    transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 21550338.94it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 52507886.36it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 10621678.02it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17461529.58it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [7]:
EPOCHS = 1
GAMMA = 0.7
LR = 1.0
log_interval = 100
TRAIN_BATCH_SIZE = 8


In [8]:
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=TRAIN_BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1000)

In [9]:
print('EPOCHS: ', EPOCHS)
print('TRAIN_BATCH_SIZE: ', TRAIN_BATCH_SIZE)

EPOCHS:  1
TRAIN_BATCH_SIZE:  8


In [21]:
model = Net().to(mps_device)

optimizer = optim.Adadelta(model.parameters(), lr=1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, EPOCHS + 1):
    train(model, mps_device, train_loader, optimizer, epoch, log_interval)
    test(model, mps_device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307632
Train Epoch: 1 [800/60000 (1%)]	Loss: 0.387665
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.581059
Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.376538
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.559387
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.265974
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.350856
Train Epoch: 1 [5600/60000 (9%)]	Loss: 0.493957
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.025418
Train Epoch: 1 [7200/60000 (12%)]	Loss: 0.257634
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.031638
Train Epoch: 1 [8800/60000 (15%)]	Loss: 0.002474
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.070965
Train Epoch: 1 [10400/60000 (17%)]	Loss: 0.001258
Train Epoch: 1 [11200/60000 (19%)]	Loss: 0.337892
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.005097
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.014632
Train Epoch: 1 [13600/60000 (23%)]	Loss: 0.004159
Train Epoch: 1 [14400/60000 (24%)]	Loss: 0.420767
Train Epoch: 1 [15200/60000 (25%)]	Loss: 0.015468
Train Epoch: 1 [16000/600